In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, root_mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Đọc dữ liệu
df = pd.read_csv("datasets/du_lieu_diem_thi_2017_2024.csv")

# Đổi tên cột cho dễ làm việc
df.columns = ['nam', 'ma_tinh', 'sbd', 'toan', 'van', 'ly', 'hoa', 'sinh', 'su', 'dia', 'gdcd', 'nn', 'ma_nn', 'tong_diem', 'khoi_a', 'khoi_a1', 'khoi_b', 'khoi_c', 'khoi_d']

# Chuyển các cột điểm về kiểu số
cols_diem = ['toan', 'van', 'ly', 'hoa', 'sinh', 'su', 'dia', 'gdcd', 'nn', 'tong_diem', 'khoi_a', 'khoi_a1', 'khoi_b', 'khoi_c', 'khoi_d']
df[cols_diem] = df[cols_diem].apply(pd.to_numeric, errors = 'coerce')

# Tạo cột tỉnh từ mã tỉnh
ma_tinh = {
    '01': 'Hà Nội', '02': 'TP. Hồ Chí Minh', '03': 'Hải Phòng', '04': 'Đà Nẵng',
    '05': 'Hà Giang', '06': 'Cao Bằng', '07': 'Lai Châu', '08': 'Lào Cai',
    '09': 'Tuyên Quang', '10': 'Lạng Sơn', '11': 'Bắc Kạn', '12': 'Thái Nguyên',
    '13': 'Yên Bái', '14': 'Sơn La', '15': 'Phú Thọ', '16': 'Vĩnh Phúc',
    '17': 'Quảng Ninh', '18': 'Bắc Giang', '19': 'Bắc Ninh', '21': 'Hải Dương',
    '22': 'Hưng Yên', '23': 'Hòa Bình', '24': 'Hà Nam', '25': 'Nam Định',
    '26': 'Thái Bình', '27': 'Ninh Bình', '28': 'Thanh Hóa', '29': 'Nghệ An',
    '30': 'Hà Tĩnh', '31': 'Quảng Bình', '32': 'Quảng Trị', '33': 'Thừa Thiên-Huế',
    '34': 'Quảng Nam', '35': 'Quảng Ngãi', '36': 'Kon Tum', '37': 'Bình Định',
    '38': 'Gia Lai', '39': 'Phú Yên', '40': 'Đắk Lắk', '41': 'Khánh Hòa',
    '42': 'Lâm Đồng', '43': 'Bình Phước', '44': 'Bình Dương', '45': 'Ninh Thuận',
    '46': 'Tây Ninh', '47': 'Bình Thuận', '48': 'Đồng Nai', '49': 'Long An',
    '50': 'Đồng Tháp', '51': 'An Giang', '52': 'Bà Rịa-Vũng Tàu', '53': 'Tiền Giang',
    '54': 'Kiên Giang', '55': 'Cần Thơ', '56': 'Bến Tre', '57': 'Vĩnh Long',
    '58': 'Trà Vinh', '59': 'Sóc Trăng', '60': 'Bạc Liêu', '61': 'Cà Mau',
    '62': 'Điện Biên', '63': 'Đắk Nông', '64': 'Hậu Giang'
}

df['ma_tinh_str'] = df['ma_tinh'].astype(str).str.zfill(2)
df['tinh'] = df['ma_tinh_str'].map(ma_tinh)

In [10]:
print("Số dòng ban đầu:", len(df))

# Loại bỏ dòng không có điểm Toán, Văn, Ngoại ngữ (3 Môn thi bắt buộc nên nếu không có thì khả năng cao là không hợp lệ)
df = df[df[['toan', 'van', 'nn']].notna().all(axis = 1)]
print("Số dòng sau khi loại bỏ dòng không có điểm Toán, Văn, Ngoại ngữ:", len(df))

# Lọc học sinh không có tổng điểm
df = df[df['tong_diem'].notna()]
print(f"Số dòng sau khi lọc học sinh không có tổng điểm: {len(df)}")

# Kiểm tra và loại bỏ dòng trùng lặp
df = df.drop_duplicates()
print(f"Số dòng sau khi loại bỏ dòng trùng lặp: {len(df)}")

Số dòng ban đầu: 7609969
Số dòng sau khi loại bỏ dòng không có điểm Toán, Văn, Ngoại ngữ: 6629910
Số dòng sau khi lọc học sinh không có tổng điểm: 6579989
Số dòng sau khi loại bỏ dòng trùng lặp: 6569125
